In [7]:
from pathlib import Path
import numpy as np
import rasterio.enums
from geotools import reproject_using_grid, georef_data_array
from products.filenames import get_datetime_from_viirs_nasa_filepath
from grids import Grid, UTM375mGrid
from pyproj import CRS
import glob
import rasterio
import xarray as xr
from datetime import timedelta

meteofrance_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/EOFR62/VIIRS2024"
nasa_geometry_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m"
nasa_geometry_reprojected_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_GEO_250m"
nasa_geometry_files = sorted(glob.glob(f"{nasa_geometry_folder}/*.nc"))

meteofrance_sample_file = glob.glob(f"{meteofrance_folder}/*.LT")[0]
dst_sample = rasterio.open(meteofrance_sample_file)
dst_shape = (dst_sample.height, dst_sample.width)

meteofrance_grid = Grid(resolution=dst_sample.transform.a,x0=dst_sample.transform.c,y0=dst_sample.transform.f,width=dst_sample.width, height=dst_sample.height, crs=dst_sample.crs)
dst_transform = dst_sample.transform
dst_crs = {'init': 'EPSG:4326'}
destination = np.zeros(dst_shape, np.uint8)



# Same as processed NASA geoemtry 
nasa_grid = UTM375mGrid()
while len(nasa_geometry_files)>0:
    print("Processing file", nasa_geometry_files[0])

    file = nasa_geometry_files[0]
    file_plus_1=nasa_geometry_files[1]
    if get_datetime_from_viirs_nasa_filepath(file_plus_1) - get_datetime_from_viirs_nasa_filepath(file) == timedelta(minutes=6):
        granule = xr.open_dataset(file)
        granule_plus_1 = xr.open_dataset(file_plus_1)
        source = xr.where(granule == 255, granule_plus_1, granule)
        nasa_geometry_files.remove(file)
        nasa_geometry_files.remove(file_plus_1)

    else:
        source = xr.open_dataset(file)
        nasa_geometry_files.remove(file)

    data_vars_reproj = {}
    for data_var_name in source:
        data_var_to_reproject =source[data_var_name].where(source[data_var_name]!=255, np.nan)
        reprojected_data_var = reproject_using_grid(dataset=georef_data_array(data_var_to_reproject,
                                                                              data_array_name=data_var_name, 
                                                                              crs=nasa_grid.crs),
                                                    output_grid=meteofrance_grid,
                                                    resampling_method=rasterio.enums.Resampling.nearest,
                                                    nodata=np.nan)
        
        
        data_vars_reproj[data_var_name] = reprojected_data_var.data_vars[data_var_name]
    reprojected = xr.Dataset(data_vars_reproj)

    reprojected.to_netcdf(f"{nasa_geometry_reprojected_folder}/{Path(file).name.replace("UTM_375m", "GEO_250m")}")

Processing file /home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m/VNP03IMG_UTM_375m.A2023274.1054.002.2023274182908.nc
Processing file /home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m/VNP03IMG_UTM_375m.A2023274.1236.002.2023274200136.nc
Processing file /home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m/VNP03IMG_UTM_375m.A2023274.1418.002.2023274224932.nc
Processing file /home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m/VNP03IMG_UTM_375m.A2023275.1036.002.2023275174816.nc
Processing file /home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m/VNP03IMG_UTM_375m.A2023275.1218.002.2023275210827.nc


KeyboardInterrupt: 

In [4]:
for i in range(5):
    print(i)

0
1
2
3
4


In [8]:
xr.open_dataset(nasa_geometry_files[0]).attrs['']

<xarray.Dataset> Size: 74MB
Dimensions:        (y: 2200, x: 2800)
Coordinates:
  * y              (y) float64 18kB 5.4e+06 5.399e+06 ... 4.576e+06 4.575e+06
  * x              (x) float64 22kB 187.5 562.5 937.5 ... 1.049e+06 1.05e+06
Data variables:
    height         (y, x) float32 25MB ...
    sensor_zenith  (y, x) float32 25MB ...
    solar_zenith   (y, x) float32 25MB ...

In [20]:
meteofrance_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/EOFR62/VIIRS2024"
meteofrance_sample_file = glob.glob(f"{meteofrance_folder}/*.LT")[0]
dst_sample = rasterio.open(meteofrance_sample_file)
dst_sample.transform

Affine(0.0024999999999999996, 0.0, -5.0025,
       0.0, -0.0025, 51.4975)

In [4]:
nasa_geometry_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m"
nasa_geometry_files = glob.glob(f"{nasa_geometry_folder}/*.nc")


In [11]:
import xarray as xr

ds = xr.open_dataset(nasa_geometry_files[0])
for name in ds:
    print(ds[name].coords['y'].attrs)

{'axis': 'Y'}
{'axis': 'Y'}
{'axis': 'Y'}


In [13]:
ds.height

<xarray.DataArray 'height' (y: 2200, x: 2800)> Size: 25MB
[6160000 values with dtype=float32]
Coordinates:
  * y        (y) float64 18kB 5.4e+06 5.399e+06 ... 4.576e+06 4.575e+06
  * x        (x) float64 22kB 187.5 562.5 937.5 ... 1.049e+06 1.049e+06 1.05e+06
Attributes:
    grid_mapping:  spatial_ref

In [2]:
source = rasterio.open(nasa_geometry_files[0])

In [3]:
source.

Affine(0.0024999999999999996, 0.0, -5.0025,
       0.0, -0.0025, 51.4975)